[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Loio-co/loio-metrics/blob/master/notebooks/Loio_metrics_demo.ipynb)

## Install the loio_nlp package

In [1]:
!python -V
!pip install git+https://github.com/Loio-co/loio-metrics.git>null

Python 3.6.9
  Running command git clone -q https://github.com/Loio-co/loio-metrics.git /tmp/pip-req-build-dxt8wkwb


In [1]:
import loio_metrics as lm
lm.version()

'0.0.2'

## Check trivial cases

To make the demo more visual, we use LTF (Loio Tagging Format). It is an XML with two tags:

- &lt;body&gt; as a root tag
- &lt;e&gt; as bounds for tagged entities

In [3]:
# Exact matching
expected = '<body>To <e>be</e> or not to <e>be</e>.</body>'
actual = '<body>To <e>be</e> or not to <e>be</e>.</body>'

expected_txt, expected_spans = lm.parse_ltf(expected)
actual_txt, actual_spans = lm.parse_ltf(actual)

# We should work with the same text
assert expected_txt == actual_txt

expected_spans, actual_spans

([Span(start=3, finish=5), Span(start=16, finish=18)],
 [Span(start=3, finish=5), Span(start=16, finish=18)])

Check if we have two true positives and zero false positives/negatives:

In [4]:
lm.calc_counters_with_partial(actual_spans, expected_spans)

Counters(tp=2.0, fp=0.0, fn=0.0)

Make this comparer as function:

In [3]:
def metrics_for_ltf(expected, actual, partial_stimulation=0.75):
  expected_txt, expected_spans = lm.parse_ltf(expected)
  actual_txt, actual_spans = lm.parse_ltf(actual)

  # We should work with the same text
  assert expected_txt == actual_txt
  return lm.calc_counters_with_partial(actual_spans, expected_spans, partial_stimulation)

Total mismatch:

In [6]:
# We should have TP=0, FP=2 and FN=1
expected = '<body>To be or <e>not</e> to be.</body>'
actual = '<body>To <e>be</e> or not to <e>be</e>.</body>'

metrics_for_ltf(expected, actual)

Counters(tp=0.0, fp=2.0, fn=1.0)

## Lets start with partial matches

In [7]:
expected = '<body>To be or <e>not</e> to be.</body>'
actual   = '<body>To be or <e>not to</e> be.</body>'

# Classical approach
print("Partial stimulation factor = 0:", metrics_for_ltf(expected, actual, partial_stimulation=0.0))

# Default value
print("Partial stimulation factor = 0.75:", metrics_for_ltf(expected, actual, partial_stimulation=0.75))

# Minimal penalty for partial matches
print("Partial stimulation factor = 1:", metrics_for_ltf(expected, actual, partial_stimulation=1.))

Partial stimulation factor = 0: Counters(tp=0.0, fp=1.0, fn=1.0)
Partial stimulation factor = 0.75: Counters(tp=0.375, fp=0.625, fn=0.625)
Partial stimulation factor = 1: Counters(tp=0.5, fp=0.5, fn=0.5)


## Tricky cases
We use partial_stimulation=1 to simplify the manual checking

In [8]:
# Only the first overlapping 3/15=0.2 is accounted to ban span separating.
expected = '<body>To <e>be or not to be</e>.</body>'
actual   = '<body>To be or <e>not</e> <e>to</e> <e>be</e>.</body>'

print(metrics_for_ltf(expected, actual, partial_stimulation=1))

Counters(tp=0.2, fp=2.8, fn=0.8)


In [9]:
# And vice versa
expected = '<body>To be or <e>not</e> <e>to</e> <e>be</e>.</body>'
actual   = '<body>To <e>be or not to be</e>.</body>'

print(metrics_for_ltf(expected, actual, partial_stimulation=1))

Counters(tp=0.2, fp=0.8, fn=2.8)


## Example from the article

In [5]:
expected = (
"<body>This DIAGNOSTIC PLATFORM BENCHMARKING STUDY AND EVALUATION AGREEMENT"
" (the “Agreement”) is made and entered into as of the last date of"
" signature below (the “Effective Date”), by and between <e>ProYard"
" Services, a Delaware corporation having its principal place of"
" business at 2140 Science Center Drive Burley, San Diego, CA, USA, ID"
" 83318 (“PartyA”)</e> and <e>ZOEMENS AG, a German corporation having its"
" principal place of business at Nuernbergerstrasse 89, 23626 Ratekau"
" (“PartyB”)</e>.</body>"
)

actual = (
"<body><e>This DIAGNOSTIC PLATFORM BENCHMARKING STUDY AND EVALUATION AGREEMENT"
" (the “Agreement”)</e> is made and entered into as of the last date of"
" signature below (the “Effective Date”), by and between <e>ProYard"
" Services, a Delaware corporation having its principal place of"
" business at 2140 Science Center Drive Burley, San Diego, CA, USA</e>, ID"
" <e>83318 (“PartyA”)</e> and <e>ZOEMENS AG, a German corporation having its"
" principal place of business at Nuernbergerstrasse 89, 23626 Ratekau"
" (“PartyB”)</e>.</body>"
)

In [6]:
counters = metrics_for_ltf(expected, actual)
print(counters)

Counters(tp=1.6490384615384617, fp=2.3509615384615383, fn=0.3509615384615383)


In [7]:
precision = counters.tp / (counters.tp + counters.fp)
recall = counters.tp / (counters.tp + counters.fn)
f1 = 2 * (precision * recall) / (precision + recall)
precision, recall, f1

(0.4122596153846154, 0.8245192307692308, 0.5496794871794872)